In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Read Diagnostic Data
data = pd.read_csv('ADNIMERGE.csv')

/Users/jnachbar/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# We only care about ID, Exam Data, Baseline, and Current
dx_data = data[['PTID', 'EXAMDATE', 'DX_bl', 'DX']]

In [4]:
# We only want Diagnoses which have changed
m = dx_data.query('DX != DX_bl')

In [5]:
# Convert to numpy so I can actually work with it
arr = m.to_numpy()

In [10]:
# If we changed from MCI to AD
last = ''
count = 0
arr_s = []
for val in arr:
    if val[0] == last:
        count += 1
    else:
        count = 0
        last = val[0]
    if val[2] == 'LMCI' and val[3] == 'Dementia':
        # if this is the case, scan backwards to figure out how many images we have before and add that to val
        tmp = list(val)
        tmp.append(count)
        arr_s.append(tmp)

In [40]:
# only capture the first instance, so the breakthrough Diagnosis
arr_final_AD = []
last_name = ''
for val in arr_s:
    if val[0] != last_name:
        arr_final_AD.append(list(val))
        last_name = val[0]     

In [41]:
# measure how many previous visits each patient has
count_arr = np.zeros(10)
for val in arr_final_AD:
    count_arr[val[4]] += 1
    
print(count_arr)

[211.  80.  86.  44.  31.  14.   5.   3.   3.   0.]


In [ ]:
# no need to include redundant data
for i in range(len(arr_final_AD)):
    arr_final_AD[i] = [arr_final_AD[i][0], arr_final_AD[i][1], arr_final_AD[i][4]]

In [45]:
# Save it as a csv
pd.DataFrame(arr_final_AD).to_csv("./MCI_to_AD.csv")

In [62]:
# capture all patients who started as MCI and remained as MCI
# take all patients who started as MCI, and subtract arr_f from it
MCI_arr = dx_data.to_numpy()
arr_m = []
for val in MCI_arr:
    if val[2] == 'LMCI':
        arr_m.append(list(val))
        

In [79]:
# go through each value
# if we get dementia or nan, skip everything on the current ID, and do not append
# otherwise, when we change, append

# x is whatever nan is in the original array. Super hacky, but whatever
x = arr_m[4991][3]

arr_final_MCI = []
count = 0
last = arr_m[0][0]
append = True
for val in arr_m:
    if val[3] in ['Dementia', x]:
        append = False
    if val[0] != last:
        if append:
            tmp = list(val)
            tmp.append(count)
            arr_final_MCI.append(tmp)
        append = True
        last = val[0]
        count = 0
    count += 1
    

In [80]:
# measure how many previous visits each patient has
count_arr = np.zeros(10)
for val in arr_final_MCI:
    count_arr[val[4]] += 1
    
print(count_arr)

[  0. 339.  46.  34.  36.  35.  24.  10.   5.   0.]


In [81]:
# no need to include redundant data
for i in range(len(arr_final_MCI)):
    arr_final_MCI[i] = [arr_final_MCI[i][0], arr_final_MCI[i][1], arr_final_MCI[i][4]]

In [82]:
pd.DataFrame(arr_final_MCI).to_csv("./MCI_to_MCI.csv")